# Writing CUDA Kernels
-------------------
## Contents
- Example - computing pairwise distances given a dataset of points
    - Solution using serial for-loops
    - Simple CUDA solution
    - Optimized CUDA solution
- Example - matrix multiplication
    - Simple CUDA solution
    - Optimized CUDA solution
-------------------

Let's see the GPU allocated to us using the ```nvidia-smi``` shell command.

In [1]:
!nvidia-smi

Wed Nov  2 14:11:09 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000035:04:00.0 Off |                    0 |
| N/A   39C    P0    35W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from numba import cuda
cuda.detect()

Found 1 CUDA devices
id 0    b'Tesla V100-SXM2-16GB'                              [SUPPORTED]
                      compute capability: 7.0
                           pci device id: 0
                              pci bus id: 4
Summary:
	1/1 devices are supported


True

### ** Switch to the `BIOE-488-v2` jupyter kernel **

In [3]:
from numba import njit, cuda, vectorize, guvectorize, stencil
from numba import prange
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import convolve as scipy_convolve

%matplotlib inline

In [4]:
# from numba.core.errors import NumbaPerformanceWarning
# import warnings
# warnings.simplefilter('ignore', category=NumbaPerformanceWarning)

---------------------
# [Section 1] Example - Computing Pairwise Distances

This computation shows up in many practical scenarios/problems:
- Distance matrices for clustering algorithms
- k-Nearest Neighbors (kNN) classification algorithm
- Estimation of topological manifolds
- Similarity-based searches such as recommendations or information retrieval

![](./distance_matrix.jpg)

Notice that:
- The cells of the distance matrix can be filled in parallel since the distance computation (Euclidean distance, for example) has no external dependencies
- Typically, the distance between A and B is the same as B and A. Let's ignore this property for now.

In [5]:
n_samples = 1000
xy_coordinates = np.random.rand(n_samples, 2)

In [6]:
xy_coordinates

array([[0.73853732, 0.23642104],
       [0.86074276, 0.39490418],
       [0.98584768, 0.83800459],
       ...,
       [0.4910636 , 0.86815911],
       [0.44461157, 0.56629412],
       [0.44235231, 0.02568889]])

---------------------
# [Section 2] Serial Solution using For-loops

In [7]:
from scipy.spatial import distance

def distance_matrix_serial(data):
    distance_matrix = np.zeros((data.shape[0], data.shape[0]))
    for i in range(data.shape[0]):
        for j in range(data.shape[0]):
            distance_matrix[i, j] = distance.euclidean(data[i], data[j])
    return distance_matrix

In [8]:
import time

start_time = time.time()
print("--- [Serial] Starting the timer ---")
result_serial = distance_matrix_serial(xy_coordinates)
print("--- Done: The execution took %s seconds ---" % (time.time() - start_time))

--- [Serial] Starting the timer ---
--- Done: The execution took 12.0077223777771 seconds ---


---------------------
# [Section 3] Simple CUDA Solution

In [9]:
from numba import cuda
from scipy.spatial import distance

# one block computes one cell inside the distance matrix
# single thread inside the block performs the computation

@cuda.jit
def distance_matrix_cuda(data, distance_matrix):

    i, j = cuda.grid(2)

    if i > len(data) or j > len(data):
        return
    
    distance_matrix[i, j] = ((data[i,0] - data[j,0])**2 + (data[i,1] - data[j,1])**2)**0.5

    return

In [10]:
import time

num_blocks = (n_samples, n_samples)
num_threads_per_block = 1

result_cuda = np.zeros((n_samples, n_samples), dtype=float)
distance_matrix_cuda[num_blocks, num_threads_per_block](xy_coordinates, result_cuda); cuda.synchronize()

start_time = time.time()
print("\n--- [simple CUDA] Starting the timer ---")
result_cuda = np.zeros((n_samples, n_samples), dtype=float)
distance_matrix_cuda[num_blocks, num_threads_per_block](xy_coordinates, result_cuda); cuda.synchronize()
print("--- Done: The execution took %s seconds ---" % (time.time() - start_time))

print("\nYour result is {}".format("correct!" if np.allclose(result_serial, result_cuda) else "incorrect."))


--- [simple CUDA] Starting the timer ---
--- Done: The execution took 0.005281209945678711 seconds ---

Your result is correct!


---------------------
# [Section 4] Simple CUDA Solution + Shared CUDA Memory

In [13]:
# one block computes distances from current point to all other points
# each point is handled by a separate thread inside the block
# threads inside the block take the current point from shared memory of the block

from numba import cuda, float64

@cuda.jit
def distance_matrix_cuda_optimized(data, distance_matrix):

    current_point = cuda.shared.array(shape=(2), dtype=float64)

    i = cuda.blockIdx.x
    if i < data.shape[0]:
        current_point = data[i]
    
    cuda.syncthreads()

    j = cuda.threadIdx.x
    if j < data.shape[0]:
        distance_matrix[i, j] = ((current_point[0] - data[j,0])**2 + (current_point[1] - data[j,1])**2) ** 0.5

    return

In [14]:
import time

num_blocks = n_samples
num_threads_per_block = n_samples

result_cuda_optimized = np.zeros((n_samples, n_samples), dtype=float)
distance_matrix_cuda_optimized[num_blocks, num_threads_per_block](xy_coordinates, result_cuda_optimized); cuda.synchronize()

start_time = time.time()
print("\n--- [simple CUDA + shared memory] Starting the timer ---")
result_cuda_optimized = np.zeros((n_samples, n_samples), dtype=float)
distance_matrix_cuda_optimized[num_blocks, num_threads_per_block](xy_coordinates, result_cuda_optimized); cuda.synchronize()
print("--- Done: The execution took %s seconds ---" % (time.time() - start_time))

print("\nYour result is {}".format("correct!" if np.allclose(result_serial, result_cuda_optimized) else "incorrect."))


--- [simple CUDA + shared memory] Starting the timer ---
--- Done: The execution took 0.004450559616088867 seconds ---

Your result is correct!


### Seeing the difference with and without shared memory more clearly (100 repeats)...

In [15]:
def repeat_simple_cuda(n_samples=1000, n_repeats=100):
    exec_times = []
    for _ in range(n_repeats):
        xy_coordinates = np.random.rand(n_samples, 2)
        num_blocks = (n_samples, n_samples)
        num_threads_per_block = 1
        result_cuda = np.zeros((n_samples, n_samples), dtype=float)
        start_time = time.time()
        distance_matrix_cuda[num_blocks, num_threads_per_block](xy_coordinates, result_cuda); cuda.synchronize()
        exec_times.append(time.time() - start_time)
    return exec_times

def repeat_optimized_cuda(n_samples=1000, n_repeats=100):
    exec_times = []
    for _ in range(n_repeats):
        xy_coordinates = np.random.rand(n_samples, 2)
        num_blocks = n_samples
        num_threads_per_block = n_samples
        result_cuda_optimized = np.zeros((n_samples, n_samples), dtype=float)
        start_time = time.time()
        distance_matrix_cuda_optimized[num_blocks, num_threads_per_block](xy_coordinates, result_cuda_optimized); cuda.synchronize()
        exec_times.append(time.time() - start_time)
    return exec_times

In [16]:
exec_times_simple_cuda = repeat_simple_cuda()
exec_times_optimized_cuda = repeat_optimized_cuda()

print(f"Simple CUDA (100 repeats): {np.mean(exec_times_simple_cuda)} ({np.std(exec_times_simple_cuda)})")
print(f"Optimized CUDA (100 repeats): {np.mean(exec_times_optimized_cuda)} ({np.std(exec_times_optimized_cuda)})")

Simple CUDA (100 repeats): 0.004269917011260987 (0.0005495242950819542)
Optimized CUDA (100 repeats): 0.0027141976356506348 (0.00032631713289847224)


---------------------
# [Section 5] Matrix multiplication - Simple CUDA Solution

In [22]:
@cuda.jit
def matmul(A, B, C):
    """Perform matrix multiplication of C = A * B
    """
    row, col = cuda.grid(2)
    if row < C.shape[0] and col < C.shape[1]:
        tmp = 0.
        for k in range(A.shape[1]):
            tmp += A[row, k] * B[k, col]
        C[row, col] = tmp

In [23]:
import numpy as np
import math

TPB = 16

# Initialize the data arrays
A = np.random.rand(TPB*200, TPB*30) # random matrix
B = np.random.rand(TPB*30, TPB*100) # random matrix

C = np.zeros((A.shape[0], B.shape[1])) # output matrix

# Configure the blocks
num_threads_per_block = (TPB, TPB)
blocks_x = int(math.ceil(A.shape[0] / num_threads_per_block[0]))
blocks_y = int(math.ceil(B.shape[1] / num_threads_per_block[1]))
num_blocks = (blocks_x, blocks_y)

matmul[num_blocks, num_threads_per_block](A, B, C); cuda.synchronize()

# Check result with serial/CPU version
result_serial = np.matmul(A, B)
print("\nYour result is {}".format("correct!" if np.allclose(result_serial, C) else "incorrect."))


Your result is correct!


---------------------
# [Section 6] Matrix multiplication - Simple CUDA Solution + Shared CUDA Memory

In [26]:
from numba import cuda, float32, float64

@cuda.jit
def fast_matmul(A, B, C):
    """
    Perform matrix multiplication of C = A * B
    Each thread computes one element of the result matrix C
    """

    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    sA = cuda.shared.array(shape=(TPB, TPB), dtype=float64)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=float64)

    x, y = cuda.grid(2)
    
    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    
    if x >= C.shape[0] and y >= C.shape[1]:
        # Quit if (x, y) is outside of valid C boundary
        return

    # Each thread computes one element in the result matrix.
    # The dot product is chunked into dot products of TPB-long vectors.
    tmp = 0.
    for i in range(int(A.shape[1] / TPB)):
        # Preload data into shared memory
        sA[tx, ty] = A[x, ty + i * TPB]
        sB[tx, ty] = B[tx + i * TPB, y]

        # Wait until all threads finish preloading
        cuda.syncthreads()

        # Computes partial product on the shared memory
        for j in range(TPB):
            tmp += sA[tx, j] * sB[j, ty]

        # Wait until all threads finish computing
        cuda.syncthreads()

    C[x, y] = tmp

In [28]:
# Controls threads per block and shared memory usage.

# The computation will be done on blocks of TPBxTPB elements.
TPB = 16

# Initialize the data arrays
A = np.random.rand(TPB*200, TPB*30) # random matrix
B = np.random.rand(TPB*30, TPB*100) # random matrix

C = np.zeros((A.shape[0], B.shape[1])) # output matrix

# Configure the blocks
num_threads_per_block = (TPB, TPB)
blocks_x = int(math.ceil(A.shape[0] / num_threads_per_block[1]))
blocks_y = int(math.ceil(B.shape[1] / num_threads_per_block[0]))
num_blocks = (blocks_x, blocks_y)

fast_matmul[num_blocks, num_threads_per_block](A, B, C); cuda.synchronize()

# Check result with serial/CPU version
result_serial = np.matmul(A, B)
print("\nYour result is {}".format("correct!" if np.allclose(result_serial, C) else "incorrect."))
print(result_serial)


Your result is correct!
[[120.773513   111.18349623 120.26918948 ... 123.45017539 122.30226101
  119.52684445]
 [127.63774925 117.27620851 121.53910656 ... 124.62518513 122.652021
  126.93094208]
 [128.05175396 117.00190238 123.32849374 ... 127.94694475 128.79161761
  122.92012669]
 ...
 [123.34495941 119.6457591  120.85944957 ... 122.93689211 125.75448403
  124.1474618 ]
 [121.40520723 117.47022588 125.75199653 ... 123.92054533 127.22949691
  124.47988488]
 [120.19825041 119.03126012 123.53457496 ... 126.32816158 122.04880435
  123.82436751]]
